In [25]:
import pandas as pd
import numpy as np
from inequalipy import *
from scipy.stats import gmean
import world_bank_data as wb
import os 
os.chdir('..')
import src.data.IDH

In [27]:
import src.data.IDH

In [3]:
# edu index ajusted

df = df1.copy()
df = df[['AGEP', 'SCH', 'SCHL']]

# calcualte the mean of years of schooling
df2 = df[df['AGEP'] > 25].copy()
df2['scholing'] =  df2['SCHL']
df2.reset_index(inplace=True)

df2['scholing'].replace({3:1, 4:2, 5:3, 6:4, 7:5, 8:6, 9:7, 10:8, 11:9, 
                        12:10, 13:11, 14:12, 15:13, 15:13, 16:13, 17:13, 
                        18:13.5, 19:14, 20:15, 21:17, 22:19, 23:19, 24:23}, inplace=True)
df2['enroled'] = np.where(df2['scholing'] > 1, 1, 0)

df2['no_zero_schooling'] = 1 + df2['scholing']
gemetric = gmean(df2['no_zero_schooling'])
amean = np.mean(df2['no_zero_schooling'])
value1 = df2['scholing'].mean()


# calculate the expected years of schooling
df3 = df[df['AGEP'] < 25].copy()
df3['enrolled'] = df3['SCH'].apply(lambda x: 1 if x > 1 else 0)
df_age = df3.groupby(['AGEP'])[['AGEP','enrolled']].count()
df_age['enrolled'] = df3.groupby(['AGEP'])['enrolled'].sum()
df_age['enrollment_rate'] = df_age['enrolled'] / df_age['AGEP']
df_age = df_age.rename (columns = {'AGEP': 'count'})
df_age = df_age.reset_index()
df_age.drop([0,1,2,3,4], inplace=True)
value2 = df_age['enrollment_rate'].sum()

# calculate index
edu_value = (value1/15 + value2/18) / 2
print(f"edu idex unajusted: {edu_value}")
print(f"edu idex new: {edu_value* (1-(1-gemetric/amean))}")
print(f"geometric mean: {gemetric}")
print(f"amean: {amean}")
print(0.847-0.834)

edu idex unajusted: 0.9064166224299821
edu idex new: 0.8471229914250292
geometric mean: 13.449334301402379
amean: 14.39070866309704
0.013000000000000012


In [4]:
# get atlas df from WB
atlas_df = pd.DataFrame(wb.get_series('NY.GNP.PCAP.CD', country='PR', simplify_index=True))
atlas_df.reset_index(inplace=True)
atlas_df.rename(columns={'NY.GNP.PCAP.CD': 'atlas'}, inplace=True)
atlas_df['atlas'] = atlas_df['atlas'].astype(float)

# get gni constant df from WB
gni_df = pd.DataFrame(wb.get_series('NY.GNP.PCAP.PP.KD', country='PR', simplify_index=True))
gni_df.reset_index(inplace=True)
gni_df.rename(columns={'NY.GNP.PCAP.PP.KD': 'gni'}, inplace=True)
gni_df['gni'] = gni_df['gni'].astype(float)

# merge the two dataframes
inc_df = atlas_df.merge(gni_df, on='Year')
inc_df['income_ratio'] = inc_df['gni'] / inc_df['atlas']
inc_df['income_ratio'] = inc_df['income_ratio'].astype(float)
inc_df['Year'] = inc_df['Year'].astype(int)

# merge the income index with the pnb.csv file
pnb = pd.read_csv('/home/coder/Documents/GitHub/IDH-index/data/external/pnb.csv')
merge_df = inc_df.merge(pnb, on='Year', how='left')
merge_df = merge_df.dropna()
merge_df.reset_index(inplace=True)
merge_df.drop(['index'], axis=1, inplace=True)
merge_df['index_temp'] = merge_df['income_ratio'] * merge_df['pnb']
merge_df['index'] = (np.log(merge_df['index_temp']) - np.log(100)) / (np.log(70000)-np.log(100))
merge_df = merge_df[['Year', 'index']]
merge_df

,Year,index
0,2006,0.842312
1,2007,0.845325
2,2008,0.839527
3,2009,0.831542
4,2010,0.825205
5,2011,0.822862
6,2012,0.831787
7,2013,0.833588
8,2014,0.834476
9,2015,0.837290


In [21]:
df_income = df1.copy()
df_income = df_income['PINCP']
#sort df_income
df_income = df_income.sort_values(ascending=True)
# remove values less than 1
df_income = df_income[df_income > 1]
# drop nan values
df_income = df_income.dropna()
df_income_bottom_5 = df_income[df_income <= df_income.quantile(0.005)]
max_income = df_income_bottom_5.max()
df_income.max()


597100.0

In [20]:
dev = df_income.copy()
# reset index
dev['PINCP'] = dev['PINCP'].apply(lambda x: max_income if x <= max_income)

300.0